This notebook exists to get the framework for how the data should be cleaned for each pokemon. Once that is done, either a new notebook or a new python script will be made with the code from this notebook to loop through all the pokemon and make the data for them

# Pulling and Cleaning Data from the PokemonAPI

In [8]:
# Libraries
import pokebase as pb
import requests
import json
import pandas as pd
import numpy as np

# Grab the data

Make the Dataframe. For now I am not sure how to handle the evolutionary data or the image data, so for now those columns will be omitted from the table

In [9]:
columns_needed = ["name","pokedex_id","attack", "special-attack", "defense", "special-defense", "speed", "hp", "type1", "type2", "list_of_moves", "ability_name", "height", "weight", "generations"]
pokemon_df = pd.DataFrame(columns=columns_needed)

In [10]:
pokemon_df.head()

,name,pokedex_id,attack,special-attack,defense,special-defense,speed,hp,type1,type2,list_of_moves,ability_name,height,weight,generations


## Pull Data From the API

Pull the pokemon data from the API

In [11]:
pokemon = requests.get('https://pokeapi.co/api/v2/pokemon/clefairy')
pokemon_data = json.loads(pokemon.text)
print(pokemon_data.keys())
# print(pokemon_data)

dict_keys(['abilities', 'base_experience', 'forms', 'game_indices', 'height', 'held_items', 'id', 'is_default', 'location_area_encounters', 'moves', 'name', 'order', 'past_types', 'species', 'sprites', 'stats', 'types', 'weight'])


In [12]:
pokemon_keys_remove = ["base_experience", "held_items", "location_area_encounters", "game_indices"] 

In [13]:
for item in pokemon_keys_remove:
    del pokemon_data[item]

Get the name and id of the pokemon

In [14]:
temp_dic = dict.fromkeys(columns_needed, -1) # this dictionary is used to store all the info needed that we will retrieve from each pokemon
# print(pokemon_data.keys())
print(temp_dic)

{'name': -1, 'pokedex_id': -1, 'attack': -1, 'special-attack': -1, 'defense': -1, 'special-defense': -1, 'speed': -1, 'hp': -1, 'type1': -1, 'type2': -1, 'list_of_moves': -1, 'ability_name': -1, 'height': -1, 'weight': -1, 'generations': -1}


In [15]:
# get pokemon name
info_holder = pokemon_data["species"]
print(info_holder)
pokemon_name = info_holder["name"]
temp_dic["name"] = pokemon_name
temp_dic["pokedex_id"] = pokemon_data["id"]

{'name': 'clefairy', 'url': 'https://pokeapi.co/api/v2/pokemon-species/35/'}


## Base Stats
Get the pokemon's base stats. This entry is another dictionary  with the `base_stat` which tells us the base stat, `effort` which is realted to EV/IV training, and `stat` which is another dicitonary with the name of the stat and the URL

In [16]:
info_holder = pokemon_data["stats"]
info_holder

[{'base_stat': 70,
  'effort': 2,
  'stat': {'name': 'hp', 'url': 'https://pokeapi.co/api/v2/stat/1/'}},
 {'base_stat': 45,
  'effort': 0,
  'stat': {'name': 'attack', 'url': 'https://pokeapi.co/api/v2/stat/2/'}},
 {'base_stat': 48,
  'effort': 0,
  'stat': {'name': 'defense', 'url': 'https://pokeapi.co/api/v2/stat/3/'}},
 {'base_stat': 60,
  'effort': 0,
  'stat': {'name': 'special-attack',
   'url': 'https://pokeapi.co/api/v2/stat/4/'}},
 {'base_stat': 65,
  'effort': 0,
  'stat': {'name': 'special-defense',
   'url': 'https://pokeapi.co/api/v2/stat/5/'}},
 {'base_stat': 35,
  'effort': 0,
  'stat': {'name': 'speed', 'url': 'https://pokeapi.co/api/v2/stat/6/'}}]

In [17]:
for entry in info_holder: # get the base stat information
    stat_name = entry["stat"]["name"]
    temp_dic[stat_name] = entry["base_stat"]            

In [18]:
temp_dic

{'name': 'clefairy',
 'pokedex_id': 35,
 'attack': 45,
 'special-attack': 60,
 'defense': 48,
 'special-defense': 65,
 'speed': 35,
 'hp': 70,
 'type1': -1,
 'type2': -1,
 'list_of_moves': -1,
 'ability_name': -1,
 'height': -1,
 'weight': -1,
 'generations': -1}

# Pokemon Type Information

In [19]:
# Get the type information of a pokemon
# pokemon_data["types"]
two_types = False
count = 0
for poke_type in pokemon_data["types"]:
    count += 1
    t = "type" + str(count)
    temp_dic[t] = poke_type["type"]["name"]
    
if count == 1: # if a pokemon only has one type, then make the second item NaN
    temp_dic["type2"] = np.nan

In [20]:
temp_dic

{'name': 'clefairy',
 'pokedex_id': 35,
 'attack': 45,
 'special-attack': 60,
 'defense': 48,
 'special-defense': 65,
 'speed': 35,
 'hp': 70,
 'type1': 'fairy',
 'type2': nan,
 'list_of_moves': -1,
 'ability_name': -1,
 'height': -1,
 'weight': -1,
 'generations': -1}

In [21]:
temp_dic["height"] = pokemon_data["height"]
temp_dic["weight"] = pokemon_data["weight"]

In [22]:
temp_dic

{'name': 'clefairy',
 'pokedex_id': 35,
 'attack': 45,
 'special-attack': 60,
 'defense': 48,
 'special-defense': 65,
 'speed': 35,
 'hp': 70,
 'type1': 'fairy',
 'type2': nan,
 'list_of_moves': -1,
 'ability_name': -1,
 'height': 6,
 'weight': 75,
 'generations': -1}

# Move Infomration
For the pokemon moves we are just storing the following information for each pokemon:
    - a list of all the moves that the pokemon can learn from level-up
    - only incldue things up to Gen 5 (Black, White, Black 2, and White 2)
    
Indicator for Black and white: `black-white`

We will be making a list of tuples. Each tuplue will will have a string that is the name of the move, and an integer which is the level that the pokemon can learn the move

In [54]:
# eploration cell, try and figure out how this dictionary is structured
# print(pokemon_data["moves"][0])
print(pokemon_data["moves"][0].keys())
print(pokemon_data["moves"][0]["move"])
print(pokemon_data["moves"][0]["version_group_details"][0].keys())
print("length of list in version_group_details", len(pokemon_data["moves"][0]["version_group_details"]))
print(pokemon_data["moves"][0]["version_group_details"][10]["version_group"])
print(pokemon_data["moves"][0]["version_group_details"][10]["level_learned_at"])

dict_keys(['move', 'version_group_details'])
{'name': 'pound', 'url': 'https://pokeapi.co/api/v2/move/1/'}
dict_keys(['level_learned_at', 'move_learn_method', 'version_group'])
length of list in version_group_details 20
{'name': 'black-white', 'url': 'https://pokeapi.co/api/v2/version-group/11/'}
1


In [62]:
# moves by level up
latest_game = "black-white"
learnable_moves = []
for move in pokemon_data["moves"]:
    name = move["move"]["name"]
    level_learned = -1
    
    for version in move["version_group_details"]: ## loop through all the game versions until you get to black and white, then use that infomration
        # print(version["move_learn_method"])
        if version["version_group"]["name"] == latest_game and version["move_learn_method"]["name"] == "level-up":
            level_learned = version["level_learned_at"]
            break
    
    if level_learned != -1:
        learnable_moves.append((name, level_learned))

In [63]:
learnable_moves

[('pound', 1),
 ('double-slap', 10),
 ('growl', 1),
 ('sing', 7),
 ('minimize', 19),
 ('defense-curl', 13),
 ('light-screen', 46),
 ('metronome', 34),
 ('encore', 4),
 ('moonlight', 40),
 ('follow-me', 16),
 ('meteor-mash', 55),
 ('cosmic-power', 28),
 ('gravity', 37),
 ('wake-up-slap', 22),
 ('healing-wish', 49),
 ('lucky-chant', 31),
 ('after-you', 52),
 ('stored-power', 43),
 ('bestow', 25)]

In [64]:
temp_dic["list_of_moves"] = learnable_moves

In [65]:
temp_dic

{'name': 'clefairy',
 'pokedex_id': 35,
 'attack': 45,
 'special-attack': 60,
 'defense': 48,
 'special-defense': 65,
 'speed': 35,
 'hp': 70,
 'type1': 'fairy',
 'type2': nan,
 'list_of_moves': [('pound', 1),
  ('double-slap', 10),
  ('growl', 1),
  ('sing', 7),
  ('minimize', 19),
  ('defense-curl', 13),
  ('light-screen', 46),
  ('metronome', 34),
  ('encore', 4),
  ('moonlight', 40),
  ('follow-me', 16),
  ('meteor-mash', 55),
  ('cosmic-power', 28),
  ('gravity', 37),
  ('wake-up-slap', 22),
  ('healing-wish', 49),
  ('lucky-chant', 31),
  ('after-you', 52),
  ('stored-power', 43),
  ('bestow', 25)],
 'ability_name': -1,
 'height': 6,
 'weight': 75,
 'generations': -1}

## Abilities Information
We will only be getting the name of the abilities and if they are hidden.

the tuples in the list will look like `(str, bool)` wheret the string is the name of the ability and bool is whether the ability is a hidden ability or not. `True` means that the ability is hidden and `False` means that the ability is not hidden.

In [68]:
# eploration cell, try and figure out how this dictionary is structured
# print(pokemon_data["abilities"][0])
print(pokemon_data["abilities"])
print(len(pokemon_data["abilities"]))

[{'ability': {'name': 'cute-charm', 'url': 'https://pokeapi.co/api/v2/ability/56/'}, 'is_hidden': False, 'slot': 1}, {'ability': {'name': 'magic-guard', 'url': 'https://pokeapi.co/api/v2/ability/98/'}, 'is_hidden': False, 'slot': 2}, {'ability': {'name': 'friend-guard', 'url': 'https://pokeapi.co/api/v2/ability/132/'}, 'is_hidden': True, 'slot': 3}]
3


In [73]:
ability_names = pd.read_csv("ability_data.csv")
ability_names = ability_names["name"].tolist()

In [75]:
possible_abilities = []
for ability in pokemon_data["abilities"]:
    if ability["ability"]["name"] in ability_names:
        name = ability["ability"]["name"]
        is_hidden = ability["is_hidden"]

        possible_abilities.append((name, is_hidden))    

In [76]:
print(possible_abilities)

[('cute-charm', False), ('magic-guard', False), ('friend-guard', True)]


In [77]:
temp_dic["ability_name"] = possible_abilities

In [ ]:
temp_dic

{'name': 'clefairy',
 'pokedex_id': 35,
 'attack': 45,
 'special-attack': 60,
 'defense': 48,
 'special-defense': 65,
 'speed': 35,
 'hp': 70,
 'type1': 'fairy',
 'type2': nan,
 'list_of_moves': [('pound', 1),
  ('double-slap', 10),
  ('growl', 1),
  ('sing', 7),
  ('minimize', 19),
  ('defense-curl', 13),
  ('light-screen', 46),
  ('metronome', 34),
  ('encore', 4),
  ('moonlight', 40),
  ('follow-me', 16),
  ('meteor-mash', 55),
  ('cosmic-power', 28),
  ('gravity', 37),
  ('wake-up-slap', 22),
  ('healing-wish', 49),
  ('lucky-chant', 31),
  ('after-you', 52),
  ('stored-power', 43),
  ('bestow', 25)],
 'ability_name': [('cute-charm', False),
  ('magic-guard', False),
  ('friend-guard', True)],
 'height': 6,
 'weight': 75,
 'generations': -1}

# Get the Generation Information

In [40]:
# data = pd.DataFrame(columns=["name", "url"], index=[0])

In [41]:
# test["results"][1]
# # x["name"] = list(x["name"])
# # x["url"] = list(x["url"])

{'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'}

In [36]:
# for pokemon in test["results"]:
#     cop = pokemon
#     cop["name"] = list(cop["name"])
#     cop["url"] = list(cop["url"])
#     temp = pd.DataFrame.from_dict(cop)
#     data = data.append(temp)

ValueError: All arrays must be of the same length